In [1]:
from sqlalchemy import create_engine ,Integer, String, Date,VARCHAR,text
import pandas as pd

In [2]:

pd.set_option('display.max_columns', None)
caminho_do_arquivo = r"D:\engenharia de dados\curso udemy\Arquivos\01.+Postgree\01. Postgree\Origem de dados\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')
colunas = ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrência", "Data_da_Ocorrencia","Municipio","UF","Regiao","Nome_do_Fabricante","Modelo"]
df = df[colunas]
df.rename( columns={  'Classificacao_da_Ocorrência' : 'Classificacao_da_Ocorrencia'  } ,inplace=True )


In [3]:
df.head()

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA,AW109SP
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT,A152
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
3,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
4,7757,Incidente Grave,2018-03-18,TORRES,RS,Sul,PIPER AIRCRAFT,PA-34-200


In [6]:
df.shape[0]

13011

Filtering by the last 3 months, but we don't have data to filter by the last 3 months, so I put a fixed variable to follow the course

In [9]:
from datetime import datetime, timedelta

data_atual = datetime.now()
data_ha_3_meses = data_atual - timedelta(days=3*30) 
data_ha_3_meses_apenas_data = data_ha_3_meses.date()

data_ha_3_meses_apenas_data = datetime(2023, 10, 18).date()

print(data_ha_3_meses_apenas_data)

2023-10-18


In [10]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

data_atual2 = datetime.now()
d_menos3M = data_atual - relativedelta(months=3) 
dt_d_menos3M = d_menos3M.date()

dt_d_menos3M = datetime(2023, 10, 16).date()

print(dt_d_menos3M)


2023-10-16


In [11]:
df.dtypes

Numero_da_Ocorrencia            int64
Classificacao_da_Ocorrencia    object
Data_da_Ocorrencia             object
Municipio                      object
UF                             object
Regiao                         object
Nome_do_Fabricante             object
Modelo                         object
dtype: object

In [12]:
df['Data_da_Ocorrencia']    = pd.to_datetime(df['Data_da_Ocorrencia'])
data_ha_3_meses_apenas_data = pd.to_datetime(data_ha_3_meses_apenas_data)
dt_d_menos3M                = pd.to_datetime(dt_d_menos3M)

In [19]:
df.shape

(537, 8)

In [18]:
df = df[df['Data_da_Ocorrencia'] >= dt_d_menos3M]

In [21]:
df.head(3)

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo
4125,39891,Incidente,2023-10-30,SOROCABA,SP,Sudeste,BEECH AIRCRAFT,58
4128,41028,Incidente,2023-11-06,CONFINS,MG,Sudeste,AIRBUS S.A.S.,A320-251N
4129,41112,Incidente,2023-11-10,JOINVILLE,SC,Sul,ATR - GIE AVIONS DE TRANSPORT RÉGIONAL,ATR-72-212A (600)


In [24]:
#conexao com banco de dados
dbname   = 'python'
user     = 'postgres'
password = 'teste123'
host     = 'localhost'
port     = '5432' 

conexao_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(conexao_str)

nome_tabela = 'anac_sqlalchemy' 

# Deletar registros com base no ano atual
cursor=engine.connect() 
delete = text(f'''    delete from public.{nome_tabela}	
                      WHERE "Data_da_Ocorrencia" >=  '{dt_d_menos3M}'
                ''')
cursor.execute(delete)
cursor.commit()

# Enviar DataFrame para o banco de dados
df.to_sql(nome_tabela, engine, index=False, if_exists='append',
                     dtype={ 
                           'Numero_da_Ocorrencia' :   Integer ,
                           'Classificacao_da_Ocorrencia': VARCHAR(50),
                           'Data_da_Ocorrencia':Date  
                           })

engine.dispose()
cursor.close()